## Import Libraries

In [ ]:
#!pip  install python-geoip
#!pip install python-geoip-geolite2
#!pip install maxminddb-geolite2
#!pip install IP2Location
#!pip install ip2geotools
!pip install plotly==4.14.3

     |████████████████████████████████| 13.2MB 303kB/s 
  Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1


In [ ]:
import csv
import time
import sys
import os.path
from os import path
from pathlib import Path
import pandas as pd
import re
import urllib.parse as urlparse
from urllib.parse import unquote
from urllib.parse import parse_qs
import matplotlib.pyplot as plt
import plotly.graph_objects as go

'''
from geolite2 import geolite2
import IP2Location
import requests
import json 
from ip2geotools.databases.noncommercial import DbIpCity
'''

'\nfrom geolite2 import geolite2\nimport IP2Location\nimport requests\nimport json \nfrom ip2geotools.databases.noncommercial import DbIpCity\n'

## Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Injection Lookup Lists

In [ ]:
sqlInjectionCmd = [" UNION "," SELECT ", " ORDER BY ", " WHERE "," CONCAT ", " union ", " select ", " order by ", " where ", " concat "]
cmdInjectionCmd = ["EXEC", "exec", "cmd", "chmod", "chown", "chgrp", "shell", "pwd", "ls", "whoami", "phpinfo()", "ipconfig", "exec", "system", "netstat" "cat", "curl", "sleep", "wget", "ping", "which", "etc/", "passwd", "md5("]
urlComponents=["User-Agent","Host","Connection","Content-Length","Accept-Encoding","Accept","Content-Type","Origin","Action","Authorization","Referer"]
safeListFile = open("/content/drive/MyDrive/logs_analysis/safelist/"+ inputFile +"_safeList_URL.txt")
safeList_URL = safeListFile.read().splitlines()

## Functions

In [ ]:
def isMalicious(rawLoad):
    x = rawLoad.split("\\r\\n")
    userAgent = "-"
    method = ""
    url = ""
    payload="-"
    comment = "Nonmalicious"
    malicious = "No"

    #find user-agent
    for i in x:
      value = re.findall(r"(?<=\: ).*", i)
      if i.lower().find("user-agent") !=-1:
        userAgent = value[0]

    if userAgent.find("gobuster/3.1.0") != -1:
      return ("Yes", "Link Brute Force", userAgent)
        
    i=x[0]
    if(i.find("b\'")!=-1 or i.find("b\"")!=-1):
        url = i
        if(i.find("GET") != -1):
            if(re.search("HTTP",url)):
                url = re.findall(r"(?<=GET )(.*)(?= HTTP)",url)[0]
                if url.find("/_ignition/execute-solution") != -1:
                  return ("Yes", "Link Brute Force", userAgent)
            else:
                url = re.findall(r"(?<=GET )(.*)",url)[0]
            method = "GET"
        elif(i.find("POST") != -1):
            if(re.search("HTTP",url)):
                  url = re.findall(r"(?<=POST )(.*)(?= HTTP)",url)[0]
                  if url.find("/_ignition/execute-solution") != -1:
                    return ("Yes", "Link Brute Force", userAgent)
            else:
                  url = re.findall(r"(?<=POST )(.*)",url)[0]
            method = "POST"
        elif(i.find("CONNECT") != -1):
            url = re.findall(r"(?<=CONNECT )(.*)(?= HTTP)",url)[0]
            method = "CONNECT"
        elif(i.find("HEAD") != -1):
            url = re.findall(r"(?<=HEAD )(.*)(?= HTTP)",url)[0]
            method = "HEAD"
        else:
            return ("Unidentified", "Unidentified","-")

   
    # find payload
    possible_payload= x[-1]
    isPayloadFound = False
    for comp in urlComponents:
        if possible_payload.find(comp) != -1:
          isPayloadFound = True
          break
    if isPayloadFound == False:
      payload = possible_payload

    # attacks   
    if userAgent.find("sqlmap") != -1:
        malicious = "Yes"
        comment="SQL Injection"
        userAgent = "Sqlmap"
        return (malicious, comment,userAgent)
    # research centers
    elif userAgent.find("CensysInspect/1.1") != -1:
        malicious = "No"
        comment="Research"
        userAgent = "Censys Research"
        return (malicious, comment,userAgent)
    elif userAgent.find("NetSystemsResearch") != -1:
        malicious = "No"
        comment="Research"
        userAgent = "NetSystems Research"
        return (malicious, comment,userAgent)
    elif userAgent.find("masscan") != -1:
        malicious = "No"
        comment="Research"
        userAgent = "Masscan Research"
        return (malicious, comment,userAgent)
    elif userAgent.find("Palo Alto") != -1:
        malicious = "No"
        comment="Research"
        userAgent = "Palo Alto Research"
        return (malicious, comment,userAgent)
    elif userAgent.find("Project-Resonance") != -1:
        malicious = "No"
        comment="Research"
        userAgent = "Project-Resonance Research"
        return (malicious, comment,userAgent)
    # webcrawl agent
    elif userAgent.find("zgrab") != -1:
        malicious = "No"
        comment="Research"
        userAgent = "Zgrab Webcrawl"    
        return (malicious, comment,userAgent)
    elif userAgent.find("Baiduspider") != -1:
        malicious = "No"
        comment="Crawler"
        userAgent = "Baidu Webcrawl"
        return (malicious, comment,userAgent)
    elif userAgent.find("Googlebot") != -1:
        malicious = "No"
        comment="Crawler"
        userAgent = "Googlebot Webcrawl"
        return (malicious, comment,userAgent)

    elif method == "POST" or method== "GET":        
        parsed = urlparse.urlparse(url)
        parsed_qs = unquote(parsed.query) 

        if parsed_qs.find("<script>") != -1 or payload.find("<script>")!=-1:
            malicious = "Yes"
            comment = "XSS"
            return (malicious, comment, userAgent)
        for cmd in sqlInjectionCmd:
            if parsed_qs.find(cmd) != -1:
                malicious = "Yes"
                comment = "SQL Injection"
                return (malicious, comment,userAgent)
        for cmd in cmdInjectionCmd:
            if url.find(cmd) != -1 or payload.find(cmd)!=-1:
                malicious = "Yes"
                comment = "Command Injection"
                return (malicious, comment,userAgent)
         
    if malicious != "Yes":      # if still not classified as malicious and yet the url is not in safe list
        if url == "/" :
           return ("No","Nonmalicious",userAgent)

        for safeUrl in safeList_URL:
            if  re.fullmatch(safeUrl,url)!=None:
              return ("No","Nonmalicious",userAgent)

        return ("Yes","Link Brute Force",userAgent)
               
        
    return (malicious, comment,userAgent)


## Generate Labeled Attacks

In [ ]:
inputFiles=["news1", "news2", "news3", "business1", "business2", "blog1"]
colNames = ["time","ip.src","tcp.srcport","ip.dst","tcp.dstport","raw.load"]

In [ ]:
for inputFile in inputFiles:
  data = pd.read_csv("/content/drive/MyDrive/logs_analysis/combined/"+ inputFile + "_combined.csv", delimiter=",", usecols=colNames )

  safeListFile = open("/content/drive/MyDrive/logs_analysis/safelist/"+ inputFile +"_safeList_URL.txt")
  safeList_URL = safeListFile.read().splitlines()

  outPath = "/content/drive/MyDrive/logs_analysis/rule/"+inputFile+"_rule.csv"
  outFile = open(outPath, "w+")
  outFile.write("time\tip.src\ttcp.srcport\tip.dst\ttcp.dstport\traw.load\tMalicious\tComment\tUser_Agent"+"\n")

  for index in range (0,len(data)):
    malicious, comment ,userAgent = isMalicious(data["raw.load"][index])
    
    #ip=data["ip.src"][index] 
    #location=getting_ip(ip)
    outFile.write( str(data["time"][index]) + "\t" + str(data["ip.src"][index]) + "\t" +
                      str(data["tcp.srcport"][index]) + "\t" +  str(data["ip.dst"][index]) + "\t" +
                      str(data["tcp.dstport"][index]) + "\t" + str(data["raw.load"][index]) + "\t" +
                      malicious + "\t" + comment + "\t" + userAgent +"\n")
    
  outFile.close()

## Analysis

In [ ]:
## merge files
allDF = []
for inputFile in inputFiles:
  print(inputFile)
  colNames = ["time","ip.src","tcp.srcport","ip.dst","tcp.dstport","raw.load","Malicious","Comment","User_Agent"]
  dataToMerge = pd.read_csv("/content/drive/MyDrive/logs_analysis/rule/"+ inputFile + "_rule.csv", delimiter="\t", usecols=colNames, error_bad_lines=False, engine ='python')
  allDF.append(dataToMerge)

combined_csv = pd.concat(allDF)
combined_csv.to_csv("/content/drive/MyDrive/logs_analysis/rule/all_data.csv", index=False, sep="\t")

news1
news2
news3
business1


Skipping line 1838: '	' expected after '"'
Skipping line 619: '	' expected after '"'
Skipping line 620: '	' expected after '"'
Skipping line 621: '	' expected after '"'
Skipping line 9262: '	' expected after '"'
Skipping line 9264: '	' expected after '"'


business2
blog1


Skipping line 185186: field larger than field limit (131072)


In [ ]:
## read files
inputFile = inputFiles[5]
print(inputFile)
colNames = ["time","ip.src","tcp.srcport","ip.dst","tcp.dstport","raw.load","Malicious","Comment","User_Agent"]
dataAnalyze = pd.read_csv("/content/drive/MyDrive/logs_analysis/rule/"+ inputFile + "_rule.csv", delimiter="\t", usecols=colNames, error_bad_lines=False, engine ='python' )
allData = pd.read_csv("/content/drive/MyDrive/logs_analysis/rule/all_data.csv", delimiter="\t", usecols=colNames, error_bad_lines=False, engine ='python')

blog1


Skipping line 185186: field larger than field limit (131072)


In [ ]:
dataAnalyze.head()

,time,ip.src,tcp.srcport,ip.dst,tcp.dstport,raw.load,Malicious,Comment,User_Agent
0,"Thursday, 29/04/21, 08:44:38 AM",194.195.112.102,40206,172.18.0.3,80,b'GET /.env HTTP/1.1\r\nHost: 104.198.59.38\r\...,Yes,Link Brute Force,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...
1,"Thursday, 29/04/21, 08:44:38 AM",194.195.112.102,40494,172.18.0.3,80,b'POST / HTTP/1.1\r\nHost: 104.198.59.38\r\nCo...,No,Nonmalicious,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...
2,"Thursday, 29/04/21, 08:45:22 AM",114.119.150.85,13174,172.18.0.3,80,b'GET /robots.txt HTTP/1.1\r\nAccept: text/htm...,Yes,Link Brute Force,Mozilla/5.0 (compatible;PetalBot;+https://webm...
3,"Tuesday, 27/04/21, 07:39:23 PM",209.17.96.210,53118,172.18.0.3,80,b'GET / HTTP/1.0\r\nUser-Agent: Mozilla/5.0 (c...,No,Nonmalicious,Mozilla/5.0 (compatible; Nimbostratus-Bot/v1.3...
4,"Tuesday, 27/04/21, 08:12:09 PM",139.162.145.250,45922,172.18.0.3,80,b'\x16\x03\x01\x00u\x01\x00\x00q\x03\x03Q>\xea...,Unidentified,Unidentified,-


In [ ]:
news1_counts = []
news1_labels = []
for item in dataAnalyze.Comment.value_counts().items():
  if(item[0].find("Unidentified")==-1):
    news1_labels.append(item[0])
    news1_counts.append(item[1])

fig = go.Figure([go.Bar(x=news1_labels, y=news1_counts)])
fig.update_layout(
    title='Blog #1 Network Traffic Classification',
    xaxis_tickfont_size=15,
    yaxis=dict(
        title='Counts',
        titlefont_size=16,
        tickfont_size=14,
    ),
    legend=dict(
        x=0,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    width=1000,
    barmode='group',
    bargap=0.15, # gap between bars of adjacent location coordinates.
    bargroupgap=0.5, # gap between bars of the same location coordinate.
)
fig.update_traces(texttemplate='%{y:.2s}', textposition='outside')
fig.show()

In [ ]:
allData_counts = []
allData_labels = []
for item in allData.Comment.value_counts().items():
    if(item[0].find("Unidentified")==-1):
      allData_labels.append(item[0])
      allData_counts.append(item[1])

fig = go.Figure([go.Bar(x=allData_labels, y=allData_counts)])
fig.update_layout(
    title='Classification of Network Traffic of All Honeypot Websites',
    xaxis_tickfont_size=15,
    yaxis=dict(
        title='Counts',
        titlefont_size=16,
        tickfont_size=14,
    ),
    legend=dict(
        x=0,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    width=1000,
    barmode='group',
    bargap=0.15, # gap between bars of adjacent location coordinates.
    bargroupgap=0.5, # gap between bars of the same location coordinate.
)
fig.update_traces(texttemplate='%{y:.2s}', textposition='outside')
fig.show()

In [ ]:
#TOOL: sqlInjection --> sqlmap 
researchData = allData.loc[allData['Comment'] == "Research"]
researchData_counts = []
researchData_labels = []
for item in researchData.User_Agent.value_counts().items():
  if(item[0].find("Mozilla")==-1):
    researchData_labels.append(item[0])
    researchData_counts.append(item[1])

fig = go.Figure([go.Bar(x=researchData_labels, y=researchData_counts)])
fig.update_layout(
    title='Classification of Research Institutes Traffic',
    xaxis_tickfont_size=15,
    yaxis=dict(
        title='Counts',
        titlefont_size=16,
        tickfont_size=14,
    ),
    legend=dict(
        x=0,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    width=1000,
    height=600,
    barmode='group',
    bargap=0.15, # gap between bars of adjacent location coordinates.
    bargroupgap=0.5, # gap between bars of the same location coordinate.
)
fig.update_traces(texttemplate='%{y:.2s}', textposition='outside')
fig.show()

### Classification of Network Traffic of All Honeypot Websites Excluding SEC509 Attack

In [ ]:
allData['time'] = pd.to_datetime(allData['time'])

In [ ]:
start_date = '04-12-2021'
end_date = '04-27-2021'
mask = (allData['time'] < start_date) | (allData['time'] > end_date)
df = allData.loc[mask]
df

,time,ip.src,tcp.srcport,ip.dst,tcp.dstport,raw.load,Malicious,Comment,User_Agent
2,2021-05-04 23:09:36,45.148.10.45,56336,172.18.0.3,80,b'POST /goform/SystemCommand HTTP/1.1\r\nHost:...,Yes,Link Brute Force,Mozilla/4.0
3,2021-05-04 23:10:07,61.219.11.153,65262,172.18.0.3,80,b'GET / HTTP/1.1\r\n\r\n',No,Nonmalicious,-
8,2021-06-04 08:41:30,121.5.103.58,35444,172.18.0.3,80,b'GET /huoshan.php HTTP/1.1\r\nHost: 34.68.176...,Yes,Link Brute Force,Mozilla/5.0 (Linux; Android 10; LIO-AN00 Build...
9,2021-06-04 08:48:00,103.113.107.90,45808,172.18.0.3,80,b'GET / HTTP/1.1\r\nHost: 34.68.176.158:80\r\n...,No,Nonmalicious,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6...
10,2021-06-04 08:48:02,172.18.0.3,38924,65.8.48.211,80,b'GET /wp/2020/03/AP_20027607175456-1000x667.j...,Yes,Link Brute Force,-
...,...,...,...,...,...,...,...,...,...
268172,2021-10-05 19:16:14,35.191.10.72,43886,172.20.0.3,80,b'GET / HTTP/1.1\r\nHost: 10.128.0.2\r\nUser-A...,No,Nonmalicious,GoogleHC/1.0
268173,2021-10-05 19:16:15,35.191.2.88,45806,172.20.0.3,80,b'GET / HTTP/1.1\r\nHost: 10.128.0.2\r\nUser-A...,No,Nonmalicious,GoogleHC/1.0
268174,2021-10-05 19:16:44,35.191.2.94,36674,172.20.0.3,80,b'GET / HTTP/1.1\r\nHost: 10.128.0.2\r\nUser-A...,No,Nonmalicious,GoogleHC/1.0
268175,2021-10-05 19:16:44,35.191.10.72,43888,172.20.0.3,80,b'GET / HTTP/1.1\r\nHost: 10.128.0.2\r\nUser-A...,No,Nonmalicious,GoogleHC/1.0


In [ ]:
df_counts = []
df_labels = []
for item in df.Comment.value_counts().items():
    if(item[0].find("Unidentified")==-1):
      df_labels.append(item[0])
      df_counts.append(item[1])

fig = go.Figure([go.Bar(x=df_labels, y=df_counts)])
fig.update_layout(
    title='Classification of Network Traffic of All Honeypot Websites Excluding SEC509 Attacks',
    xaxis_tickfont_size=15,
    yaxis=dict(
        title='Counts',
        titlefont_size=16,
        tickfont_size=14,
    ),
    legend=dict(
        x=0,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    width=1000,
    barmode='group',
    bargap=0.15, # gap between bars of adjacent location coordinates.
    bargroupgap=0.5, # gap between bars of the same location coordinate.
)
fig.update_traces(texttemplate='%{y:.2s}', textposition='outside')
fig.show()